In [1]:
import pandas as pd
import gspread
import matplotlib as mat
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pa_functions as pa

In [2]:
creds = pa.get_creds(r'C:\Users\joao.batista\Desktop\scripts\token.json')

In [3]:
df = pa.get_sheets(creds, '1TiH9ZRvsCuhI71fD5WH1vVsvj2kEVTjgIkUx1FIl4tQ', 'base')
df.head()

,DATA,NOME LOOKER,ID,NOME SUP,LOJA,VALOR REFERÊNCIA,VALOR COMISSIONAMENTO,GMV TOTAL,META,GMV vs META,SHARE 1P,SHARE PL,FAIXA,RANGE,% DE COMISSÃO
1,01/01/2024,ANA PAULA MOURA SANTOS,1_5738,VANDERSON MENEZES MORENO SILVA,"07 - GS - Centro - São Caetano, SP","1.101,54","0,00","97.484,80","162.903,98",60%,68%,32%,F-,0% - 60%,0%
2,01/01/2024,JOSE OTAVIO LEMES,1_5947,THAIS SIMS RODRIGUES SIQUEIRA,"108 - GS - Centro - Londrina, PR","706,91","0,00","56.557,38","122.628,73",46%,79%,35%,F-,0% - 60%,0%
3,01/01/2024,JOICE BARROS,1_6081,THAIS SIMS RODRIGUES SIQUEIRA,"108 - GS - Centro - Londrina, PR","497,41","0,00","42.384,18","122.628,73",35%,67%,34%,F-,0% - 60%,0%
4,01/01/2024,LENNON GUILHERME DE OLIVEIRA,1_5949,MARIANA AKEMI NAKANISHI DO NASCIMENTO,"12 - GS - Conceição - Campinas, SP","267,39","0,00","17.659,11","135.000,00",13%,96%,43%,F+,0% - 60%,0%
5,01/01/2024,CAMILA THOMPSON,1_6161,PRISCILLA RATTES GONCALVES DA SILVA,"129 - GS - Centro - Paulínia, SP","1.024,74","0,00","78.468,02","140.484,40",56%,80%,38%,F-,0% - 60%,0%


In [5]:
df['DATA'] = pd.to_datetime(df['DATA'],dayfirst=True,infer_datetime_format=True)
df['MES'] = df['DATA'].dt.month
df['MES'].value_counts()

MES
2    231
1    214
3    199
4    195
Name: count, dtype: int64

In [6]:
df[['VALOR REFERÊNCIA','VALOR COMISSIONAMENTO','GMV TOTAL','META']] = df[['VALOR REFERÊNCIA','VALOR COMISSIONAMENTO','GMV TOTAL','META']].replace('\.','',regex=True)
df[['VALOR REFERÊNCIA','VALOR COMISSIONAMENTO','GMV TOTAL','META']] = df[['VALOR REFERÊNCIA','VALOR COMISSIONAMENTO','GMV TOTAL','META']].replace(',','.',regex=True)
df[['VALOR REFERÊNCIA','VALOR COMISSIONAMENTO','GMV TOTAL','META']] = df[['VALOR REFERÊNCIA','VALOR COMISSIONAMENTO','GMV TOTAL','META']].astype(float)

In [9]:
df.head()

,DATA,NOME LOOKER,ID,NOME SUP,LOJA,VALOR REFERÊNCIA,VALOR COMISSIONAMENTO,GMV TOTAL,META,GMV vs META,SHARE 1P,SHARE PL,FAIXA,RANGE,% DE COMISSÃO,MES
1,2024-01-01,ANA PAULA MOURA SANTOS,1_5738,VANDERSON MENEZES MORENO SILVA,"07 - GS - Centro - São Caetano, SP",1101.54,0.0,97484.80,162903.98,60%,68%,32%,F-,0% - 60%,0%,1
2,2024-01-01,JOSE OTAVIO LEMES,1_5947,THAIS SIMS RODRIGUES SIQUEIRA,"108 - GS - Centro - Londrina, PR",706.91,0.0,56557.38,122628.73,46%,79%,35%,F-,0% - 60%,0%,1
3,2024-01-01,JOICE BARROS,1_6081,THAIS SIMS RODRIGUES SIQUEIRA,"108 - GS - Centro - Londrina, PR",497.41,0.0,42384.18,122628.73,35%,67%,34%,F-,0% - 60%,0%,1
4,2024-01-01,LENNON GUILHERME DE OLIVEIRA,1_5949,MARIANA AKEMI NAKANISHI DO NASCIMENTO,"12 - GS - Conceição - Campinas, SP",267.39,0.0,17659.11,135000.00,13%,96%,43%,F+,0% - 60%,0%,1
5,2024-01-01,CAMILA THOMPSON,1_6161,PRISCILLA RATTES GONCALVES DA SILVA,"129 - GS - Centro - Paulínia, SP",1024.74,0.0,78468.02,140484.40,56%,80%,38%,F-,0% - 60%,0%,1


In [16]:
df_agrupado = df[['DATA', 'GMV TOTAL', 'VALOR REFERÊNCIA']].groupby('DATA').sum().reset_index()
df_agrupado

,DATA,GMV TOTAL,VALOR REFERÊNCIA
0,2024-01-01,2.640744e+07,375887.840000
1,2024-02-01,1.984010e+07,272512.710000
2,2024-03-01,2.219556e+07,305845.640000
3,2024-04-01,2.018302e+07,265777.889156


In [18]:
df_agrupado.dtypes

0
DATA                datetime64[ns]
GMV TOTAL                  float64
VALOR REFERÊNCIA           float64
dtype: object

In [30]:
correlation = df_agrupado['VALOR REFERÊNCIA'].corr(df_agrupado['GMV TOTAL'])
correlation

0.9949027604586

Aplicando ADF - Teste de Dickey-Fuller aumentado 

In [14]:
from statsmodels.tsa.stattools import adfuller

In [17]:
df_agrupado['DATA'] = pd.to_datetime(df_agrupado['DATA'])

In [19]:
df_agrupado.set_index('DATA', inplace=True)

In [25]:
serie_temporal = df_agrupado['VALOR REFERÊNCIA']

In [34]:
result = adfuller(serie_temporal)

In [29]:
result_summary = {
    'ADF Statistic': result[0],
    'p-value': result[1],
    'Used lags': result[2],
    'Number of observations': result[3],
    'Critical Values': result[4]
}

for key, value in result_summary.items():
    if key == 'Critical Values':
        print(f'{key}:')
        for k, v in value.items():
            print(f'   {k}: {v}')
    else:
        print(f'{key}: {value}')

ADF Statistic: -4.002704687236415
p-value: 0.0013964147473546924
Used lags: 0
Number of observations: 3
Critical Values:
   1%: -10.41719074074074
   5%: -5.77838074074074
   10%: -3.391681111111111


Tornando a série estacionária

In [47]:
import math

In [57]:
df_agrupado['GMV TOTAL LOG'] = df_agrupado['GMV TOTAL'].apply(lambda x: math.log(x))
df_agrupado

,GMV TOTAL,VALOR REFERÊNCIA,GMV TOTAL SQRT,GMV TOTAL LOG,GMV TOTAL CBRT
DATA,,,,,
2024-01-01,2.640744e+07,375887.840000,297.789098,17.089157,297.789098
2024-02-01,1.984010e+07,272512.710000,270.716447,16.803216,270.716447
2024-03-01,2.219556e+07,305845.640000,281.031756,16.915403,281.031756
2024-04-01,2.018302e+07,265777.889156,272.267216,16.820352,272.267216


In [59]:
df_agrupado['GMV TOTAL SQRT'] = df_agrupado['GMV TOTAL'].apply(lambda x: np.sqrt(x))
df_agrupado

,GMV TOTAL,VALOR REFERÊNCIA,GMV TOTAL SQRT,GMV TOTAL LOG,GMV TOTAL CBRT
DATA,,,,,
2024-01-01,2.640744e+07,375887.840000,5138.817461,17.089157,297.789098
2024-02-01,1.984010e+07,272512.710000,4454.223052,16.803216,270.716447
2024-03-01,2.219556e+07,305845.640000,4711.216861,16.915403,281.031756
2024-04-01,2.018302e+07,265777.889156,4492.551100,16.820352,272.267216


In [60]:
df_agrupado['GMV TOTAL CBRT'] = df_agrupado['GMV TOTAL'].apply(lambda x: np.cbrt(x))
df_agrupado

,GMV TOTAL,VALOR REFERÊNCIA,GMV TOTAL SQRT,GMV TOTAL LOG,GMV TOTAL CBRT
DATA,,,,,
2024-01-01,2.640744e+07,375887.840000,5138.817461,17.089157,297.789098
2024-02-01,1.984010e+07,272512.710000,4454.223052,16.803216,270.716447
2024-03-01,2.219556e+07,305845.640000,4711.216861,16.915403,281.031756
2024-04-01,2.018302e+07,265777.889156,4492.551100,16.820352,272.267216


In [62]:
df_agrupado['GMV TOTAL DIFF'] = df_agrupado['GMV TOTAL'].diff()
df_agrupado

,GMV TOTAL,VALOR REFERÊNCIA,GMV TOTAL SQRT,GMV TOTAL LOG,GMV TOTAL CBRT,GMV TOTAL DIFF
DATA,,,,,,
2024-01-01,2.640744e+07,375887.840000,5138.817461,17.089157,297.789098,NaN
2024-02-01,1.984010e+07,272512.710000,4454.223052,16.803216,270.716447,-6.567342e+06
2024-03-01,2.219556e+07,305845.640000,4711.216861,16.915403,281.031756,2.355461e+06
2024-04-01,2.018302e+07,265777.889156,4492.551100,16.820352,272.267216,-2.012549e+06


In [66]:
df_agrupado['GMV TOTAL DIFF'] = df_agrupado['GMV TOTAL DIFF'].fillna(0)
df_agrupado

,GMV TOTAL,VALOR REFERÊNCIA,GMV TOTAL SQRT,GMV TOTAL LOG,GMV TOTAL CBRT,GMV TOTAL DIFF
DATA,,,,,,
2024-01-01,2.640744e+07,375887.840000,5138.817461,17.089157,297.789098,0.000000e+00
2024-02-01,1.984010e+07,272512.710000,4454.223052,16.803216,270.716447,-6.567342e+06
2024-03-01,2.219556e+07,305845.640000,4711.216861,16.915403,281.031756,2.355461e+06
2024-04-01,2.018302e+07,265777.889156,4492.551100,16.820352,272.267216,-2.012549e+06


In [67]:
serie_temporal = df_agrupado['GMV TOTAL DIFF']

In [68]:
result = adfuller(serie_temporal)

In [70]:
result_summary = {
    'ADF Statistic': result[0],
    'p-value': result[1],
    'Used lags': result[2],
    'Number of observations': result[3],
    'Critical Values': result[4]
}

for key, value in result_summary.items():
    if key == 'Critical Values':
        print(f'{key}:')
        for k, v in value.items():
            print(f'   {k}: {v}')
    else:
        print(f'{key}: {value}')

ADF Statistic: -2.438849788630403
p-value: 0.13107014625202962
Used lags: 0
Number of observations: 3
Critical Values:
   1%: -10.41719074074074
   5%: -5.77838074074074
   10%: -3.391681111111111


In [8]:
gmv_loja = df[['MES','LOJA','GMV TOTAL','META']].groupby(['MES','LOJA']).sum().reset_index()
gmv_loja['GMV vs Meta'] = gmv_loja['GMV TOTAL'] / gmv_loja['META']
gmv_loja['LOJA'] = gmv_loja['LOJA'].str.upper()
gmv_loja = gmv_loja.rename(columns={'MES':'mes', 'LOJA':'agrupamento','GMV TOTAL':'gmv_total','META':'meta','GMV vs Meta':'gmv_vs_meta'})
gmv_loja


,mes,agrupamento,gmv_total,meta,gmv_vs_meta
0,1,"01 - GS - CENTRO - CURITIBA, PR",592085.790000,557375.12000,1.062275
1,1,"02 - GS - NOVO MUNDO - CURITIBA, PR",554228.280000,544942.08000,1.017041
2,1,"03 - GS - TATUAPÉ - SÃO PAULO, SP",309301.610000,305114.31000,1.013724
3,1,"05 - GS - LAPA - SÃO PAULO, SP",623649.080000,500000.00000,1.247298
4,1,"06 - GS - PARQUE MARIA HELENA - SÃO PAULO, SP",282023.580000,397021.48000,0.710348
...,...,...,...,...,...
215,4,"77 - GS - SAGRADA FAMÍLIA - BELO HORIZONTE, MG",521533.851100,576654.29720,0.904413
216,4,"79 - GS - BACACHERI - CURITIBA, PR",489897.960730,563939.20669,0.868707
217,4,"91 - GS - CENTRO - SÃO JOSÉ DO RIO PRETO, SP",250595.599786,322250.97523,0.777641
218,4,"94 - GS - CENTRO - SANTOS, SP",268249.409800,338426.54681,0.792637


In [84]:
substituir_letras = {
        'Á': 'A', 'À': 'A', 'Â': 'A', 'Ã': 'A',
        'É': 'E', 'È': 'E', 'Ê': 'E',
        'Í': 'I', 'Ì': 'I', 'Î': 'I',
        'Ó': 'O', 'Ò': 'O', 'Ô': 'O', 'Õ': 'O',
        'Ú': 'U', 'Ù': 'U', 'Û': 'U',
        'Ç': 'C'
    }

In [85]:
gmv_loja['agrupamento'] = gmv_loja['agrupamento'].replace(substituir_letras,regex=True)
gmv_loja['grupo'] = 'loja'
gmv_loja['key'] = gmv_loja['mes'].astype(str) + '_' + gmv_loja['agrupamento']
gmv_loja

,mes,agrupamento,gmv_total,meta,gmv_vs_meta,grupo,key
0,1,"01 - GS - CENTRO - CURITIBA, PR",592085.790000,557375.12000,1.062275,loja,"1_01 - GS - CENTRO - CURITIBA, PR"
1,1,"02 - GS - NOVO MUNDO - CURITIBA, PR",554228.280000,544942.08000,1.017041,loja,"1_02 - GS - NOVO MUNDO - CURITIBA, PR"
2,1,"03 - GS - TATUAPE - SAO PAULO, SP",309301.610000,305114.31000,1.013724,loja,"1_03 - GS - TATUAPE - SAO PAULO, SP"
3,1,"05 - GS - LAPA - SAO PAULO, SP",623649.080000,500000.00000,1.247298,loja,"1_05 - GS - LAPA - SAO PAULO, SP"
4,1,"06 - GS - PARQUE MARIA HELENA - SAO PAULO, SP",282023.580000,397021.48000,0.710348,loja,"1_06 - GS - PARQUE MARIA HELENA - SAO PAULO, SP"
...,...,...,...,...,...,...,...
215,4,"77 - GS - SAGRADA FAMILIA - BELO HORIZONTE, MG",521533.851100,576654.29720,0.904413,loja,"4_77 - GS - SAGRADA FAMILIA - BELO HORIZONTE, MG"
216,4,"79 - GS - BACACHERI - CURITIBA, PR",489897.960730,563939.20669,0.868707,loja,"4_79 - GS - BACACHERI - CURITIBA, PR"
217,4,"91 - GS - CENTRO - SAO JOSE DO RIO PRETO, SP",250595.599786,322250.97523,0.777641,loja,"4_91 - GS - CENTRO - SAO JOSE DO RIO PRETO, SP"
218,4,"94 - GS - CENTRO - SANTOS, SP",268249.409800,338426.54681,0.792637,loja,"4_94 - GS - CENTRO - SANTOS, SP"


In [86]:
gmv_coord = df[['MES','NOME SUP','GMV TOTAL','META']].groupby(['MES','NOME SUP']).sum().reset_index()
gmv_coord['GMV vs Meta'] = gmv_coord['GMV TOTAL'] / gmv_coord['META']
gmv_coord = gmv_coord.rename(columns={'MES':'mes', 'NOME SUP':'agrupamento','GMV TOTAL':'gmv_total','META':'meta','GMV vs Meta':'gmv_vs_meta'})
gmv_coord['grupo'] = 'coordenador'
gmv_coord['key'] = gmv_coord['mes'].astype(str) + '_' + gmv_coord['agrupamento']


In [87]:
gmv_coord

,mes,agrupamento,gmv_total,meta,gmv_vs_meta,grupo,key
0,1,ADRIANA CARNEIRO DA SILVA,1.111402e+06,1.409000e+06,0.788788,coordenador,1_ADRIANA CARNEIRO DA SILVA
1,1,CAROLINE YOGI CARDOSO,1.779709e+06,1.802675e+06,0.987260,coordenador,1_CAROLINE YOGI CARDOSO
2,1,CIBELE KHAIRALLA MARTINS,1.545377e+06,1.794899e+06,0.860982,coordenador,1_CIBELE KHAIRALLA MARTINS
3,1,CRISTIANE GABOR DE MIRANDA,1.788322e+06,2.242670e+06,0.797408,coordenador,1_CRISTIANE GABOR DE MIRANDA
4,1,FABIANA OLIVEIRA BAHIA,9.244616e+05,1.270000e+06,0.727923,coordenador,1_FABIANA OLIVEIRA BAHIA
...,...,...,...,...,...,...,...
63,4,RAFAEL BARBOSA,1.428988e+06,1.855723e+06,0.770044,coordenador,4_RAFAEL BARBOSA
64,4,SIMONE MARA CARVALHO DE OLIVEIRA,1.147575e+06,1.390662e+06,0.825201,coordenador,4_SIMONE MARA CARVALHO DE OLIVEIRA
65,4,THAIS SIMS RODRIGUES SIQUEIRA,1.058775e+06,1.319215e+06,0.802580,coordenador,4_THAIS SIMS RODRIGUES SIQUEIRA
66,4,VANDERSON MENEZES MORENO SILVA,1.539037e+06,2.020335e+06,0.761773,coordenador,4_VANDERSON MENEZES MORENO SILVA


In [88]:
gmv = pd.concat([gmv_loja,gmv_coord], axis=0)
gmv = gmv.drop(columns=['mes','agrupamento'])
gmv

,gmv_total,meta,gmv_vs_meta,grupo,key
0,5.920858e+05,5.573751e+05,1.062275,loja,"1_01 - GS - CENTRO - CURITIBA, PR"
1,5.542283e+05,5.449421e+05,1.017041,loja,"1_02 - GS - NOVO MUNDO - CURITIBA, PR"
2,3.093016e+05,3.051143e+05,1.013724,loja,"1_03 - GS - TATUAPE - SAO PAULO, SP"
3,6.236491e+05,5.000000e+05,1.247298,loja,"1_05 - GS - LAPA - SAO PAULO, SP"
4,2.820236e+05,3.970215e+05,0.710348,loja,"1_06 - GS - PARQUE MARIA HELENA - SAO PAULO, SP"
...,...,...,...,...,...
63,1.428988e+06,1.855723e+06,0.770044,coordenador,4_RAFAEL BARBOSA
64,1.147575e+06,1.390662e+06,0.825201,coordenador,4_SIMONE MARA CARVALHO DE OLIVEIRA
65,1.058775e+06,1.319215e+06,0.802580,coordenador,4_THAIS SIMS RODRIGUES SIQUEIRA
66,1.539037e+06,2.020335e+06,0.761773,coordenador,4_VANDERSON MENEZES MORENO SILVA


In [89]:
df1 = pa.get_sheets(creds, '10KV2Iy0z_DlrhIljAUZHjp_Rz3c1SwMRDTXTI4et0_8', 'indicadores_base')
df1.head()

,key,metric_value
1,2023_1_1_mes_desligamento_involuntario_corpora...,0
2,2023_1_1_mes_desligamento_involuntario_corpora...,0
3,2023_1_1_mes_desligamento_involuntario_corpora...,0
4,2023_1_1_mes_desligamento_involuntario_geral_g...,0
5,2023_1_1_mes_desligamento_involuntario_geral_g...,0


In [90]:
df1 = df1[~df1['key'].str.startswith('2023')]
df1 = df1[~df1['key'].str.contains('company')]
df1 = df1[~df1['key'].str.contains('CD')]
# df1 = df1[~df1['key'].str.contains('ytd')]
df1 = df1[~df1['key'].str.contains('quarter')]
df1 = df1[~df1['key'].str.contains('corporativooperacional')]
df1 = df1[~df1['key'].str.contains('tipoheadcount')]
df1 = df1[~df1['key'].str.contains('operacional')]
df1 = df1[~df1['key'].str.contains('corporativo')]
df1 = df1[~df1['key'].str.contains('lideranca')]
df1 = df1[~df1['key'].str.contains('tenureanos')]

In [91]:
df1['agrupamento'] = df1['key'].str.split('_').str[-1]
df1['mes'] = df1['key'].str.split('_').str[1]
df1['tipo'] = df1['key'].str.split('_').str[5]
df1['metrica'] = df1['key'].str.split('_').str[4]
df1['valor'] = df1['key'].str.split('_').str[3]
df1 = df1.drop(columns=['key'])



In [92]:
df1['key'] = df1['mes'].astype(str) + '_' + df1['agrupamento']


In [93]:
df1 = df1[['key','metric_value','agrupamento','mes','tipo','metrica','valor']]
df1 = df1[(df1['agrupamento'] != 'PEDRO HENRIQUE MOYA LEMOS') & (df1['agrupamento'] != 'SOFIA GUIMARAES RAMOS')]

In [94]:
df1

,key,metric_value,agrupamento,mes,tipo,metrica,valor
29480,"1_108 - GS - CENTRO - LONDRINA, PR",0,"108 - GS - CENTRO - LONDRINA, PR",1,involuntario,desligamento,mes
29481,"1_108 - GS - CENTRO - LONDRINA, PR",0,"108 - GS - CENTRO - LONDRINA, PR",1,involuntario,desligamento,mes
29482,"1_108 - GS - CENTRO - LONDRINA, PR",0,"108 - GS - CENTRO - LONDRINA, PR",1,involuntario,desligamento,mes
29483,"1_108 - GS - CENTRO - LONDRINA, PR",0,"108 - GS - CENTRO - LONDRINA, PR",1,involuntario,desligamento,mes
29500,"1_102 - GS - CENTRO - AMERICANA, SP",0,"102 - GS - CENTRO - AMERICANA, SP",1,involuntario,desligamento,mes
...,...,...,...,...,...,...,...
711777,"5_77 - GS - SAGRADA FAMILIA - BELO HORIZONTE, MG","0,8667","77 - GS - SAGRADA FAMILIA - BELO HORIZONTE, MG",5,total,turnover,ytd
711782,"5_91 - GS - CENTRO - SAO JOSE DO RIO PRETO, SP",0,"91 - GS - CENTRO - SAO JOSE DO RIO PRETO, SP",5,total,turnover,ytd
711820,"5_95 - GS - SAO MIGUEL PAULISTA - SAO PAULO, SP",0,"95 - GS - SAO MIGUEL PAULISTA - SAO PAULO, SP",5,total,turnover,ytd
711821,"5_95 - GS - SAO MIGUEL PAULISTA - SAO PAULO, SP",0,"95 - GS - SAO MIGUEL PAULISTA - SAO PAULO, SP",5,total,turnover,ytd


In [95]:
df1.reset_index()

,index,key,metric_value,agrupamento,mes,tipo,metrica,valor
0,29480,"1_108 - GS - CENTRO - LONDRINA, PR",0,"108 - GS - CENTRO - LONDRINA, PR",1,involuntario,desligamento,mes
1,29481,"1_108 - GS - CENTRO - LONDRINA, PR",0,"108 - GS - CENTRO - LONDRINA, PR",1,involuntario,desligamento,mes
2,29482,"1_108 - GS - CENTRO - LONDRINA, PR",0,"108 - GS - CENTRO - LONDRINA, PR",1,involuntario,desligamento,mes
3,29483,"1_108 - GS - CENTRO - LONDRINA, PR",0,"108 - GS - CENTRO - LONDRINA, PR",1,involuntario,desligamento,mes
4,29500,"1_102 - GS - CENTRO - AMERICANA, SP",0,"102 - GS - CENTRO - AMERICANA, SP",1,involuntario,desligamento,mes
...,...,...,...,...,...,...,...,...
24103,711777,"5_77 - GS - SAGRADA FAMILIA - BELO HORIZONTE, MG","0,8667","77 - GS - SAGRADA FAMILIA - BELO HORIZONTE, MG",5,total,turnover,ytd
24104,711782,"5_91 - GS - CENTRO - SAO JOSE DO RIO PRETO, SP",0,"91 - GS - CENTRO - SAO JOSE DO RIO PRETO, SP",5,total,turnover,ytd
24105,711820,"5_95 - GS - SAO MIGUEL PAULISTA - SAO PAULO, SP",0,"95 - GS - SAO MIGUEL PAULISTA - SAO PAULO, SP",5,total,turnover,ytd
24106,711821,"5_95 - GS - SAO MIGUEL PAULISTA - SAO PAULO, SP",0,"95 - GS - SAO MIGUEL PAULISTA - SAO PAULO, SP",5,total,turnover,ytd


In [96]:
df_final = pd.merge(df1,gmv,on='key', how='left')
df_final.head()

,key,metric_value,agrupamento,mes,tipo,metrica,valor,gmv_total,meta,gmv_vs_meta,grupo
0,"1_108 - GS - CENTRO - LONDRINA, PR",0,"108 - GS - CENTRO - LONDRINA, PR",1,involuntario,desligamento,mes,301304.71,490514.92,0.614262,loja
1,"1_108 - GS - CENTRO - LONDRINA, PR",0,"108 - GS - CENTRO - LONDRINA, PR",1,involuntario,desligamento,mes,301304.71,490514.92,0.614262,loja
2,"1_108 - GS - CENTRO - LONDRINA, PR",0,"108 - GS - CENTRO - LONDRINA, PR",1,involuntario,desligamento,mes,301304.71,490514.92,0.614262,loja
3,"1_108 - GS - CENTRO - LONDRINA, PR",0,"108 - GS - CENTRO - LONDRINA, PR",1,involuntario,desligamento,mes,301304.71,490514.92,0.614262,loja
4,"1_102 - GS - CENTRO - AMERICANA, SP",0,"102 - GS - CENTRO - AMERICANA, SP",1,involuntario,desligamento,mes,306443.19,407903.82,0.751263,loja


In [97]:
df_final['metric_value'] = df_final['metric_value'].str.replace(',','.').astype(float)



In [98]:
df_final.head()

,key,metric_value,agrupamento,mes,tipo,metrica,valor,gmv_total,meta,gmv_vs_meta,grupo
0,"1_108 - GS - CENTRO - LONDRINA, PR",0.0,"108 - GS - CENTRO - LONDRINA, PR",1,involuntario,desligamento,mes,301304.71,490514.92,0.614262,loja
1,"1_108 - GS - CENTRO - LONDRINA, PR",0.0,"108 - GS - CENTRO - LONDRINA, PR",1,involuntario,desligamento,mes,301304.71,490514.92,0.614262,loja
2,"1_108 - GS - CENTRO - LONDRINA, PR",0.0,"108 - GS - CENTRO - LONDRINA, PR",1,involuntario,desligamento,mes,301304.71,490514.92,0.614262,loja
3,"1_108 - GS - CENTRO - LONDRINA, PR",0.0,"108 - GS - CENTRO - LONDRINA, PR",1,involuntario,desligamento,mes,301304.71,490514.92,0.614262,loja
4,"1_102 - GS - CENTRO - AMERICANA, SP",0.0,"102 - GS - CENTRO - AMERICANA, SP",1,involuntario,desligamento,mes,306443.19,407903.82,0.751263,loja


In [99]:
df_final['metric_value'].value_counts()

metric_value
0.0000    20746
1.0000      947
2.0000      556
0.2500      304
0.5000      178
          ...  
8.0000        1
0.6611        1
0.5758        1
0.2912        1
0.6472        1
Name: count, Length: 90, dtype: int64

In [100]:
df_final.dtypes

0
key              object
metric_value    float64
agrupamento      object
mes              object
tipo             object
metrica          object
valor            object
gmv_total       float64
meta            float64
gmv_vs_meta     float64
grupo            object
dtype: object

In [101]:
# df_final_filtrado = df_final[(df_final['grupo'] == 'coordenador') & (df_final['metrica'] == 'turnover') & (df_final['valor'] == 'mes')]
# correlation = df_final_filtrado['metric_value'].corr(df_final_filtrado['gmv_vs_meta'])
correlation = df_final['metric_value'].corr(df_final['gmv_vs_meta'])
correlation

-0.0648177063689122

In [102]:
pa.export_to_sheets(creds,df_final,destiny_sheets=[['1TiH9ZRvsCuhI71fD5WH1vVsvj2kEVTjgIkUx1FIl4tQ','Meta X Desligamentos']])

>>> 1 df(s) exported in 13.93s
